In [ ]:
import numpy as np
import matplotlib.pyplot as pl
import ot
import ot.plot
import time

In [ ]:
# Some global figure settings
%matplotlib inline
params = {'legend.fontsize': 'large',
          'figure.figsize': (14, 7),
         'axes.labelsize': 'X-large',
         'axes.titlesize':'X-large',
         'xtick.labelsize':'large',
         'ytick.labelsize':'large',
        'axes.titlesize': 16
         }

pl.rcParams.update(params)

#Make sure the figures directory exists
try:
    os.makedirs('./figures')
except: 
    print('Figures folder exists')

# Class and functions for LOT WassMap.

A Class that encapsulates the information for an LOT Wassmap experiment with Gaussian data and reference measure. 

In [ ]:
def calc_OTmap(xr, xt, a=None, b=None, M=None, 
               sinkhorn=False, lambd=1, normalize_T=True):
    # calculates barycentric projection of OT map
    m, dr = xr.shape
    n, dt = xt.shape
    if a is None:
        a = np.ones((m,))/m
    if b is None:
        b = np.ones((n,))/n
        
    if M is None:
        assert dr == dt
        M = ot.dist(xr, xt) # cost matrix for Wasserstein distance computation
    
    if sinkhorn:
        # compute Sinkhorn plan
        G = ot.sinkhorn(a, b, M, lambd)
    else:
        # compute LP plan
        G = ot.emd(a, b, M)
    # row-normalize plan
    Gstochastic = G/G.sum(axis=1)[:,None]
    T = Gstochastic @ xt # calculate barycentric projection
    if normalize_T:
        T = T/np.sqrt(m)
    return T
    
def compute_embedding(transportMat, Ynorm=None):
    # Computes embedding with optimal rotation
    # compute mean for centering of transport matrix
    rowmean = transportMat.mean(axis=0, keepdims=True) 
    transportMatCentered = transportMat - rowmean # center transport matrix
     # SVD of centered transport matrix
    u, s, vh = np.linalg.svd(transportMatCentered, full_matrices=False)
    u = u[:,:2] @ np.diag(s[:2])
    embedding = u
    if Ynorm is not None:
        unorm = u
        innerprod = Ynorm.T @ unorm
        u2,s2,vh2 = np.linalg.svd(innerprod)
        rotation_matrix = u2 @ vh2
        # Rotate embedding
        embedding = embedding @ rotation_matrix.T
    return embedding

def calculate_embedding(xr, xt_list, Y=None, a=None, 
                        b_list=None, partial=1,lambd=1):
    # calculates all embeddings from a reference measure and list of target measures
    N = len(xt_list)
    m, d = xr.shape
    
    transportMat = np.zeros((N//partial, m*d)) # initialize matrix of transport maps
    entropictransportMat =  np.zeros((N//partial, m*d))
    #Y = np.zeros((N//partial, 2)) 
    for i in range(N//partial):
        # get data for OT map calculations
        xt = xt_list[i]
        b = None
        if b_list is not None:
            b = b_list[i]
        # calculate OT maps
        T = calc_OTmap(xr, xt, a=a, b=b, sinkhorn=False)
        entropicT = calc_OTmap(xr, xt, a=a, b=b, sinkhorn=True,lambd=1)
        # fill in transport matrix
        transportMat[i,:] = T.flatten('F')
        entropictransportMat[i,:] = entropicT.flatten('F')
    # calculate embeddings
    embedding = compute_embedding(transportMat, Ynorm=Y)
    entropicembedding = compute_embedding(entropictransportMat, Ynorm=Y)
    return embedding, entropicembedding

def calculate_embedding_LP(xr, xt_list, Y=None, a=None, 
                        b_list=None, partial=1,lambd=1):
    # calculates all embeddings from a reference measure and list of target measures
    tic = time.time()
    N = len(xt_list)
    m, d = xr.shape
    
    transportMat = np.zeros((N//partial, m*d)) # initialize matrix of transport maps
    #Y = np.zeros((N//partial, 2)) 
    for i in range(N//partial):
        # get data for OT map calculations
        xt = xt_list[i]
        b = None
        if b_list is not None:
            b = b_list[i]
        # calculate OT maps
        T = calc_OTmap(xr, xt, a=a, b=b, sinkhorn=False)
        # fill in transport matrix
        transportMat[i,:] = T.flatten('F')
    # calculate embeddings
    embedding = compute_embedding(transportMat, Ynorm=Y)
    toc = time.time()
    timing = toc-tic
    return embedding#, timing

def calculate_embedding_Sinkhorn(xr, xt_list, Y=None, a=None, 
                        b_list=None, partial=1,lambd=1):
    # calculates all embeddings from a reference measure and list of target measures
    tic = time.time()
    N = len(xt_list)
    m, d = xr.shape
    
    entropictransportMat = np.zeros((N//partial, m*d)) # initialize matrix of transport maps
    #Y = np.zeros((N//partial, 2)) 
    for i in range(N//partial):
        # get data for OT map calculations
        xt = xt_list[i]
        b = None
        if b_list is not None:
            b = b_list[i]
        # calculate OT maps
        T = calc_OTmap(xr, xt, a=a, b=b, sinkhorn=True, lambd = lambd)
        # fill in transport matrix
        entropictransportMat[i,:] = T.flatten('F')
    # calculate embeddings
    entropicembedding = compute_embedding(entropictransportMat, Ynorm=Y)
    toc = time.time()
    timing = toc-tic
    return entropicembedding#, timing


def wass_matrix(xt_list,a = None, b_list = None, squared=True,p=2.0):
    """
    The function compute the (squared if squared=True) Wasserstein Distance Matrix between N images
    image_list: python list of pointcloud representations 
    """
    N = len(xt_list) #number of images
    wass_dist = np.zeros((N,N)) #initialize the distance matrix
    for i in range(N):
        for j in range(i+1,N):
            n, dt = xt_list[i].shape
            if a is None:
                a = np.ones((n,))/n
            if b_list[i] is not None:
                b = b_list[i]
            M = np.power(ot.dist(xt_list[i], xt_list[j],'euclidean'),p)
            wass_dist[i,j] = ot.emd2(a,b,M)
            
    wass_dist += wass_dist.T

    if(squared==True):
        wass_dist = np.square(wass_dist) 
    return wass_dist

def calculate_MDS_embedding(xt_list, a = None, b_list = None, squared=True,num_components = 2):
    """
    The function computes the MDS embedding from a set of data measures.
    """
    N = len(xt_list)
    H = np.eye(N)-1/N*np.ones((N,N))
    dist_matrix = wass_matrix(xt_list, a = a, b_list = b_list)
    if squared==False:
        B = -.5*H@(dist_matrix**2)@H
    else:
        B = -.5*H@dist_matrix@H
    U,S,VT = np.linalg.svd(B)
    embedding = U[:,:num_components]@np.diag(S[:num_components]**.5)

    return embedding
    
    

class LOT_WassMap_Gauss(object):
    
    def __init__(self, m, n, N, mu_r, cov_r, mu_t_list, cov_t_list, Y=None, partial=1,
                b_list = None, lambd=1, c = pl.cm.tab10, figs = True):
        self.m = m
        self.n = n
        self.N = N
        self.a = np.ones((self.m,)) / self.m
        self.partial = 1
        self.Y = np.zeros((self.N//self.partial, 2)) 
        self.b_list = []
        self.lambd = lambd
        self.c = c(np.linspace(0,1,N))
        self.figs = figs
        # empirical sample of size m of reference measure
        self.xr = ot.datasets.make_2D_samples_gauss(m, mu_r, cov_r)
        self.xt_list = []
        # create Gauss
        for i in range(N//self.partial):
            mu_t = mu_t_list[i]
            self.Y[i,:] = mu_t
            self.b_list.append( np.ones((self.n,))/self.n )
            cov_t = cov_t_list[i]
            xt = ot.datasets.make_2D_samples_gauss(self.n, mu_t, cov_t)
            self.xt_list.append(xt)
        if Y is not None:
            self.Y = Y
        if b_list is not None:
            self.b_list = b_list
            
    def calc_LOT_WassMap(self,):
        
        # create embeddings
        self.embedding, self.entropicembedding = calculate_embedding(self.xr,
                                            self.xt_list, Y=self.Y,
                                            a=self.a, b_list=self.b_list, lambd=self.lambd,
                                            partial=self.partial)
        
        err = np.linalg.norm(self.Y-self.embedding,'fro')/np.linalg.norm(self.Y,'fro')
        entropic_error = np.linalg.norm(self.Y-self.entropicembedding)/np.linalg.norm(self.Y)
        
        # If figs is true, the data and reference distributions are plotted along with the embeddings
        if self.figs:
            fig = pl.figure(figsize=(14,7))
            ax = fig.add_subplot(141)
            # plot reference distribution
            ax.scatter(self.xr[:, 0], self.xr[:, 1], marker='+',
                       color='blue', label='Source samples')
            ax.set_title('Data')
            ax.set_aspect('equal')
            for i in range(N//partial):
                xt = self.xt_list[i]
                # add plot of data measure to existing plot
                ax.scatter(xt[:, 0], xt[:, 1], 
                           marker='x',c=[self.c[i]]*self.n,
                        label='Target samples')

            # plot reference distribution
            ax2 = fig.add_subplot(142)
            ax2.scatter(Y[:, 0], Y[:, 1], c=self.c)
            ax2.set_title('True Embedding Points')
            ax2.set_aspect('equal')


            # Plot final rotated embedding
            ax3 = fig.add_subplot(143)
            ax3.scatter(self.embedding[:, 0], self.embedding[:, 1], c=self.c)
            ax3.set_title('Linear Program Embedding')
            ax3.set_aspect('equal')

            # Plot final rotated embedding
            ax4 = fig.add_subplot(144)
            ax4.scatter(self.entropicembedding[:, 0], self.entropicembedding[:, 1], c=self.c)
            ax4.set_title('Sinkhorn Embedding')
            ax4.set_aspect('equal')
            fig.tight_layout(pad=5.0)
            pl.show()
        
        return self.embedding, self.entropicembedding, err, entropic_error
    
    def LOT_timing(self,):
        # For timing MDS vs. LOT with Linear Program vs. LOT with Sinkhorn
        tic = time.time()
        self.embedding = calculate_embedding_LP(self.xr,self.xt_list,Y=self.Y,
                                            a=self.a, b_list=self.b_list, lambd=self.lambd,
                                            partial=self.partial)
        toc = time.time()
        LP_timing = toc-tic
        
        tic = time.time()
        self.entropicembedding = calculate_embedding_Sinkhorn(self.xr,self.xt_list,Y=self.Y,
                                            a=self.a, b_list=self.b_list, lambd=self.lambd,
                                            partial=self.partial)
        toc = time.time()
        Sinkhorn_timing = toc-tic
        
        tic = time.time()
        self.MDSembedding = calculate_MDS_embedding(self.xt_list,a=self.a,b_list=self.b_list)
        toc = time.time()
        MDS_timing = toc-tic
        
        return self.embedding, LP_timing, self.entropicembedding, Sinkhorn_timing, self.MDSembedding, MDS_timing
    
    def LOT_timing_LP(self,):
        # For timing LOT with the Linear Program used to compute transport maps
        
        tic = time.time()
        self.embedding = calculate_embedding_LP(self.xr,self.xt_list,Y=self.Y,
                                            a=self.a, b_list=self.b_list, lambd=self.lambd,
                                            partial=self.partial)
        toc = time.time()
        LP_timing = toc-tic
        
        return self.embedding, LP_timing
    
    def LOT_timing_Sinkhorn(self, lambd = None):
        # For timing LOT with the Sinkhorn used to compute transport maps
        
        if lambd is None:
            lambd = self.lambd
        
        tic = time.time()
        self.entropicembedding = calculate_embedding_Sinkhorn(self.xr,self.xt_list,Y=self.Y,
                                            a=self.a, b_list=self.b_list, lambd=lambd,
                                            partial=self.partial)
        toc = time.time()
        Sinkhorn_timing = toc-tic
        
        return self.entropicembedding, Sinkhorn_timing

# 1-D Translations with Perturbations
Experiment 1 and Figures 1 and 2 from the paper. We use a standard normal reference distribution, and the data is a set of $N=10 $ Gaussians with means on a circle of radius 4, all having the same dependent covariance matrix which is contaminated with random Gaussia noise. Each measure is sampled uniformly.

Note that when the reference and data measures are sampled sparsely, the embedding will contain some scaling distortion. For larger sample sizes (>100 or so) the scale of the embedding is accurate.

In [ ]:
N = 10       # Number of data points
partial = 1  # Subsampling parameter of data points (if desired)
d = 2        # ambient data dimension
m = 1000      # Number of samples of reference measure
n = m        # Number of samples of data measures
lambd = 1 # entropic regularization parameter

#### Generate Reference Measure ####
## Reference measure is an empirical sample of N(0,I)
mu_r = np.array([0, 0])   # mean 0
cov_r = np.array([[1, 0], [0, 1]])  # Identity covariance matrix

radius = 8 # set radius governing means of data points (larger is more visible, but performance is similar for all)
noise_level = 0.5 # noise level (standard deviation) added to covariance matrix of data measures
Y = np.zeros((N//partial, 2)) 

mu_t_list = []
cov_t_list = []


for i in range(N//partial):
    mu_t = np.array([radius*np.cos(2*np.pi*(i-1)/N), radius*np.sin(2*np.pi*(i-1)/N)]) # Mean centered on circle of radius radius
    cov_t = np.array([[1, -.5], [-.5, 1]]) # correlated covariance matrix
    errMat = noise_level * np.random.rand(d,d)  # random Guassian noise
    cov_t = cov_t + errMat @ np.transpose(errMat) # Add symmetric noise to covariance matrix
    mu_t_list.append(mu_t)
    cov_t_list.append(cov_t)
    # empirical sample of size n of data measure
    Y[i,:] = mu_t

    
lot_WassMap = LOT_WassMap_Gauss(m, n, N, mu_r, cov_r, mu_t_list,
                cov_t_list, Y=Y, partial=partial,figs=True,lambd=lambd)
embedding, entropic_embedding, err, entropic_error = lot_WassMap.calc_LOT_WassMap()

Timing experiment for the circle translation example above.

In [ ]:
N = 10       # Number of data points
partial = 1  # Subsampling parameter of data points (if desired)
d = 2        # ambient data dimension
mlist = np.arange(50,850,50) # list with number of samples m of the reference measure
nlist = mlist # list with number of samples n of data measures -- default the same as m
numiter = 10  # Number of iterations (each trial of the LOT Wassmap algorithm is run numiter times and error is averaged)
lambd = 1 # entropic regularization parameter

#### Generate Reference Measure ####
## Reference measure is an empirical sample of N(0,I)
mu_r = np.array([0, 0])   # mean 0
cov_r = np.array([[1, 0], [0, 1]])  # Identity covariance matrix

radius = 8 # set radius governing means of data points (larger is more visible, but performance is similar for all)
noise_level = 0.5 # noise level (standard deviation) added to covariance matrix of data measures
Y = np.zeros((N//partial, 2)) 

mu_t_list = []
cov_t_list = []


for i in range(N//partial):
    mu_t = np.array([radius*np.cos(2*np.pi*(i-1)/N), radius*np.sin(2*np.pi*(i-1)/N)]) # Mean centered on circle of radius radius
    cov_t = np.array([[1, -.5], [-.5, 1]]) # correlated covariance matrix
    errMat = noise_level * np.random.rand(d,d)  # random Guassian noise
    cov_t = cov_t + errMat @ np.transpose(errMat) # Add symmetric noise to covariance matrix
    mu_t_list.append(mu_t)
    cov_t_list.append(cov_t)
    # empirical sample of size n of data measure
    Y[i,:] = mu_t


errStore_1dtrans = np.zeros((len(mlist),len(nlist), numiter))
errStoreEntropic_1dtrans = np.zeros((len(mlist),len(nlist), numiter))

for repiter in range(numiter):
    for miter in range(len(mlist)):
        for niter in range(len(nlist)): 

            m = mlist[miter]
            n = nlist[niter]
            
            lot_WassMap = LOT_WassMap_Gauss(m, n, N, mu_r, cov_r, mu_t_list,
                cov_t_list, Y=Y, partial=partial,figs=False)
            embedding, entropic_embedding, err, entropic_error = lot_WassMap.calc_LOT_WassMap()
            
            errStore_1dtrans[miter,niter,repiter] = err
            errStoreEntropic_1dtrans[miter,niter,repiter] = entropic_error

errMean_1dtrans = np.mean(errStore_1dtrans[0,:,:],axis=1)
errStd_1dtrans = np.std(errStore_1dtrans[0,:,:],axis=1)

Entropic_errorMean_1dtrans = np.mean(errStoreEntropic_1dtrans[0,:,:],axis=1)
Entropic_errorStd_1dtrans = np.std(errStoreEntropic_1dtrans[0,:,:],axis=1)

fig = pl.figure(figsize=(12,5))
ax = fig.add_subplot(121)
ax.plot(nlist,errMean_1dtrans,'b',marker='o')
ax.fill_between(nlist,errMean_1dtrans - errStd_1dtrans,errMean_1dtrans + errStd_1dtrans,color = 'gray',alpha=.2)
ax.set_title('Linear Program')
ax.set_xticks(np.arange(100,900,100))
ax.set_xlabel('Sample size $m$')
ax.set_ylabel('Relative Error')
# ax.set_aspect('equal')

ax2 = fig.add_subplot(122)
ax2.plot(nlist,Entropic_errorMean_1dtrans,'b',marker='o')
ax2.fill_between(nlist,Entropic_errorMean_1dtrans - Entropic_errorStd_1dtrans,Entropic_errorMean_1dtrans + Entropic_errorStd_1dtrans,color = 'gray',alpha=.2)
ax2.set_title('Sinkhorn')
ax2.set_xticks(np.arange(100,900,100))
ax2.set_xlabel('Sample size $m$')
ax2.set_ylabel('Relative Error')
#ax2.set_aspect('equal')
fig.tight_layout(pad=5.0)
pl.show()

# Rotations with Perturbations
Experiment 2 and Figures 3 and 4 from the paper. We use a standard normal reference distribution, and the data is a set of N=10 Gaussians with means on a circle of radius 8, where the dependent covariance matrices are also rotated according to the rotation angle; in this way, we form a 1-dimensional rotation manifold governed by the circle. Each measure is sampled uniformly.

Note that when the reference and data measures are sampled sparsely, the embedding will contain some scaling distortion. For larger sample sizes (>100 or so) the scale of the embedding is accurate.

In [ ]:
#### Parameters ####
N = 10      # Number of data points
partial = 1  # Subsampling parameter of data points (if desired)
d = 2        # ambient data dimension
m = 1000      # Number of samples of reference measure
n = m        # Number of samples of data measures
lambd = 1    # entropic regularization parameter

#### Generate Reference Measure ####
## Reference measure is an empirical sample of N(0,I)

a, b = np.ones((m,)) / m, np.ones((n,)) / n  # uniform distribution on samples

mu_r = np.array([0, 0])   # mean 0
cov_r = np.array([[1, 0], [0, 1]])  # Identity covariance matrix

Y = np.zeros((N//partial, 2)) 

radius = 8 # set radius governing means of data points (larger is more visible, but performance is similar for all)
noise_level = 0.5 # noise level (standard deviation) added to covariance matrix of data measures

mu_t_list = []
cov_t_list = []

## Create data measures and add to plot of reference distribution
for i in range(N//partial):
    angle = 2*np.pi*(i-1)/N   # angles uniformly sampling the unit circle
    angle_vec = np.array([np.cos(angle), np.sin(angle)])   # mean corresponds to points on the circle of radius r
    mu_t = angle_vec*radius
    cov_t = np.array([[2, 0], [0, 0.5]])   # covariance matrix
    rotation_matrix = np.array([[np.cos(angle), -np.sin(angle)],[np.sin(angle),np.cos(angle)]]) # form rotation matrix
    cov_t = rotation_matrix @ cov_t @ rotation_matrix.T  # add rotation to covariance matrix
    
    errMat = noise_level * np.random.rand(d,d) # random Gaussian noise
    cov_t = cov_t + errMat @ np.transpose(errMat) # Add symmetric noise to covariance matrix
    mu_t_list.append(mu_t)
    cov_t_list.append(cov_t)
    
    Y[i,:] = mu_t
    
lot_WassMap = LOT_WassMap_Gauss(m, n, N, mu_r, cov_r, mu_t_list,
                cov_t_list, Y=Y, partial=partial)
lot_WassMap.calc_LOT_WassMap()

Timing experiment for the rotation manifold example above.

In [ ]:
#### Parameters ####
N = 10      # Number of data points
partial = 1  # Subsampling parameter of data points (if desired)
d = 2        # ambient data dimension
mlist = np.arange(50,850,50)
nlist = mlist
numiter = 10  # Number of iterations (each trial of the LOT Wassmap algorithm is run numiter times and error is averaged)
lambd = 1    # entropic regularization parameter

#### Generate Reference Measure ####
## Reference measure is an empirical sample of N(0,I)

a, b = np.ones((m,)) / m, np.ones((n,)) / n  # uniform distribution on samples

mu_r = np.array([0, 0])   # mean 0
cov_r = np.array([[1, 0], [0, 1]])  # Identity covariance matrix

Y = np.zeros((N//partial, 2)) 

radius = 8 # set radius governing means of data points (larger is more visible, but performance is similar for all)
noise_level = 0.5 # noise level (standard deviation) added to covariance matrix of data measures

mu_t_list = []
cov_t_list = []

## Create data measures and add to plot of reference distribution
for i in range(N//partial):
    angle = 2*np.pi*(i-1)/N   # angles uniformly sampling the unit circle
    angle_vec = np.array([np.cos(angle), np.sin(angle)])   # mean corresponds to points on the circle of radius r
    mu_t = angle_vec*radius
    cov_t = np.array([[2, 0], [0, 0.5]])   # covariance matrix
    rotation_matrix = np.array([[np.cos(angle), -np.sin(angle)],[np.sin(angle),np.cos(angle)]]) # form rotation matrix
    cov_t = rotation_matrix @ cov_t @ rotation_matrix.T  # add rotation to covariance matrix
    
    errMat = noise_level * np.random.rand(d,d) # random Gaussian noise
    cov_t = cov_t + errMat @ np.transpose(errMat) # Add symmetric noise to covariance matrix
    mu_t_list.append(mu_t)
    cov_t_list.append(cov_t)
    
    Y[i,:] = mu_t

errStore = np.zeros((len(mlist),len(nlist), numiter))
errStoreEntropic = np.zeros((len(mlist),len(nlist), numiter))

for repiter in range(numiter):
    for miter in range(len(mlist)):
        for niter in range(len(nlist)): 

            m = mlist[miter]
            n = nlist[niter]
            
            lot_WassMap = LOT_WassMap_Gauss(m, n, N, mu_r, cov_r, mu_t_list,
                cov_t_list, Y=Y, partial=partial,figs=False)
            embedding, entropic_embedding, err, entropic_error = lot_WassMap.calc_LOT_WassMap()
            
            errStore[miter,niter,repiter] = err
            errStoreEntropic[miter,niter,repiter] =entropic_error

errMean = np.mean(errStore[0,:,:],axis=1)
errStd = np.std(errStore[0,:,:],axis=1)

Entropic_errorMean = np.mean(errStoreEntropic[0,:,:],axis=1)
Entropic_errorStd = np.std(errStoreEntropic[0,:,:],axis=1)

fig = pl.figure(figsize=(12,5))
ax = fig.add_subplot(121)
ax.plot(nlist,errMean,'b',marker='o')
ax.fill_between(nlist,errMean - errStd,errMean + errStd,color = 'gray',alpha=.2)
ax.set_title('Linear Program')
ax.set_xticks(np.arange(100,900,100))
ax.set_xlabel('Sample size $m$')
ax.set_ylabel('Relative Error')
# ax.set_aspect('equal')

ax2 = fig.add_subplot(122)
ax2.plot(nlist,Entropic_errorMean,'b',marker='o')
ax2.fill_between(nlist,Entropic_errorMean - Entropic_errorStd,Entropic_errorMean + Entropic_errorStd,color = 'gray',alpha=.2)
ax2.set_title('Sinkhorn')
ax2.set_xticks(np.arange(100,900,100))
ax2.set_xlabel('Sample size $m$')
ax2.set_ylabel('Relative Error')
#ax2.set_aspect('equal')
fig.tight_layout(pad=5.0)
pl.show()

# 2-D Translations with Perturbations
Experiment 3 and Figures 5 and 6 from the paper. We use a standard normal reference distribution, and the data is a set of N=10 Gaussians with means on a uniform 5x5 grid on the cube $[-10,10]^2$, all having the same dependent covariance matrix which are contaminated with random Gaussian noise. Each measure is sampled uniformly.

Note that when the reference and data measures are sampled sparsely, the embedding will contain some scaling distortion. For larger sample sizes (>100 or so) the scale of the embedding is accurate.

In [ ]:
#### Parameters ####
N = 25       # Number of data points
partial = 1  # Subsampling parameter of data points (if desired)
d = 2        # ambient data dimension
m = 1000      # Number of samples of reference measure
n = m        # Number of samples of data measures
lambd = 1e1    # entropic regularization parameter

#### Generate Reference Measure ####
## Reference measure is an empirical sample of N(0,I)

a, b = np.ones((m,)) / m, np.ones((n,)) / n  # uniform distribution on samples

mu_r = np.array([0, 0])   # mean 0
cov_r = np.array([[1, 0], [0, 1]])  # Identity covariance matrix


Y = np.zeros((N//partial, 2)) 

noise_level = 0.5 # noise level (standard deviation) added to covariance matrix of data measures

mu_t_list = []
cov_t_list = []

# form coordinate grid for the 2-d translation lattice
x = np.linspace(-10, 10, int(np.sqrt(N)))
y = np.linspace(-10, 10, int(np.sqrt(N)))
# full coordinate arrays
xx, yy = np.meshgrid(x, y)
xx = xx.reshape(-1)
yy = yy.reshape(-1)

for i in range(N//partial):
    mu_t = np.array([xx[i], yy[i]]) # mean on the sampled 2-D lattice
    cov_t = np.array([[1, -.5], [-.5, 1]]) # correlated covariance matrix
    errMat = noise_level * np.random.rand(d,d) # random Gaussian noise
    cov_t = cov_t + errMat @ np.transpose(errMat) # Add symmetric noise to covariance matrix
    
    mu_t_list.append(mu_t)
    cov_t_list.append(cov_t)
    
    Y[i,:] = mu_t
    

lot_WassMap = LOT_WassMap_Gauss(m, n, N, mu_r, cov_r, mu_t_list,
                cov_t_list, Y=Y, partial=partial, lambd=lambd, c = pl.cm.turbo)
embedding, entropic_embedding, err, entropic_error = lot_WassMap.calc_LOT_WassMap()   

Timing experiment for the grid translation example above.

In [ ]:
#### Parameters ####
N = 25       # Number of data points
partial = 1  # Subsampling parameter of data points (if desired)
d = 2        # ambient data dimension
mlist = np.arange(50,850,50)
nlist = mlist
numiter = 10  # Number of iterations (each trial of the LOT Wassmap algorithm is run numiter times and error is averaged)

lambd = 1e1    # entropic regularization parameter

#### Generate Reference Measure ####
## Reference measure is an empirical sample of N(0,I)

a, b = np.ones((m,)) / m, np.ones((n,)) / n  # uniform distribution on samples

mu_r = np.array([0, 0])   # mean 0
cov_r = np.array([[1, 0], [0, 1]])  # Identity covariance matrix


Y = np.zeros((N//partial, 2)) 

noise_level = 0.5 # noise level (standard deviation) added to covariance matrix of data measures

mu_t_list = []
cov_t_list = []

# form coordinate grid for the 2-d translation lattice
x = np.linspace(-10, 10, int(np.sqrt(N)))
y = np.linspace(-10, 10, int(np.sqrt(N)))
# full coordinate arrays
xx, yy = np.meshgrid(x, y)
xx = xx.reshape(-1)
yy = yy.reshape(-1)

for i in range(N//partial):
    mu_t = np.array([xx[i], yy[i]]) # mean on the sampled 2-D lattice
    cov_t = np.array([[1, -.5], [-.5, 1]]) # correlated covariance matrix
    errMat = noise_level * np.random.rand(d,d) # random Gaussian noise
    cov_t = cov_t + errMat @ np.transpose(errMat) # Add symmetric noise to covariance matrix
    
    mu_t_list.append(mu_t)
    cov_t_list.append(cov_t)
    
    Y[i,:] = mu_t
    
errStore_2dtrans = np.zeros((len(mlist),len(nlist), numiter))
errStoreEntropic_2dtrans = np.zeros((len(mlist),len(nlist), numiter))

for repiter in range(numiter):
    for miter in range(len(mlist)):
        for niter in range(len(nlist)): 

            m = mlist[miter]
            n = nlist[niter]
            
            lot_WassMap = LOT_WassMap_Gauss(m, n, N, mu_r, cov_r, mu_t_list,
                cov_t_list, Y=Y, partial=partial,figs=False)
            embedding, entropic_embedding, err, entropic_error = lot_WassMap.calc_LOT_WassMap()
            
            errStore_2dtrans[miter,niter,repiter] = err
            errStoreEntropic_2dtrans[miter,niter,repiter] =entropic_error

errMean_2dtrans = np.mean(errStore_2dtrans[0,:,:],axis=1)
errStd_2dtrans = np.std(errStore_2dtrans[0,:,:],axis=1)

Entropic_errorMean_2dtrans = np.mean(errStoreEntropic_2dtrans[0,:,:],axis=1)
Entropic_errorStd_2dtrans = np.std(errStoreEntropic_2dtrans[0,:,:],axis=1)

fig = pl.figure(figsize=(12,5))
ax = fig.add_subplot(121)
ax.plot(nlist,errMean_2dtrans,'b',marker='o')
ax.fill_between(nlist,errMean_2dtrans - errStd_2dtrans,errMean_2dtrans + errStd_2dtrans,color = 'gray',alpha=.2)
ax.set_title('Linear Program')
ax.set_xticks(np.arange(100,900,100))
ax.set_xlabel('Sample size $m$')
ax.set_ylabel('Relative Error')
# ax.set_aspect('equal')

ax2 = fig.add_subplot(122)
ax2.plot(nlist,Entropic_errorMean_2dtrans,'b',marker='o')
ax2.fill_between(nlist,Entropic_errorMean_2dtrans - Entropic_errorStd_2dtrans,Entropic_errorMean_2dtrans + Entropic_errorStd_2dtrans,color = 'gray',alpha=.2)
ax2.set_title('Sinkhorn')
ax2.set_xticks(np.arange(100,900,100))
ax2.set_xlabel('Sample size $m$')
ax2.set_ylabel('Relative Error')
#ax2.set_aspect('equal')
fig.tight_layout(pad=5.0)
pl.show()

## Anisotropic Dilation

Experiment 4 and Figures 7 and 8 from the paper. We use a standard normal reference distribution, and the data is a set of $N=10 $ Gaussians with mean 0 and scaled covariance matrices which are contaminated with random Gaussian noise. Each measure is sampled uniformly.

Note that when the reference and data measures are sampled sparsely, the embedding will contain some scaling distortion. For larger sample sizes (>100 or so) the scale of the embedding is accurate.

In [ ]:
#### Parameters ####
N = 9       # Number of data points
partial = 1  # Subsampling parameter of data points (if desired)
d = 2        # ambient data dimension
m = 1000      # Number of samples of reference measure
n = 2500       # Number of samples of data measures

#### Generate Reference Measure ####
## Reference measure is an empirical sample of N(0,I)

a, b = np.ones((m,)) / m, np.ones((n,)) / n  # uniform distribution on samples

mu_r = np.array([0, 0])   # mean 0
cov_r = np.array([[1, 0], [0, 1]]) # reference covariance matrix

Y = np.zeros((N//partial, 2)) 

noise_level = 0.5 #.5 # noise level (standard deviation) added to covariance matrix of data measures

mu_t_list = []
cov_t_list = []

scaling = np.linspace(1, 4, num=int(np.sqrt(N)))
scaling = np.flip(scaling)
XX, YY = np.meshgrid(scaling,scaling)
XX = XX.reshape(-1)
YY = YY.reshape(-1)

for i in range(N//partial):
    mu_t = np.array([0, 0])
    #cov_t = np.array([[1, -.5], [-.5, 1]])
    cov_t = np.array([[1, 0], [0, 1]])
    errMat = noise_level * np.random.rand(d,d)
    cov_t = cov_t + errMat @ np.transpose(errMat)
    cov_t = np.diag([XX[i],YY[i]])@cov_t@np.diag([XX[i],YY[i]])
    
    mu_t_list.append(mu_t)
    cov_t_list.append(cov_t)

    Y[i,0] = XX[i] - np.mean(XX)
    Y[i,1] = YY[i] - np.mean(YY)

lot_WassMap = LOT_WassMap_Gauss(m, n, N, mu_r, cov_r, mu_t_list,
                cov_t_list, Y=Y, partial=partial, lambd = 1e2)
embedding, entropic_embedding, err, entropic_error = lot_WassMap.calc_LOT_WassMap()    

Timing experiment for the dilation manifold above. Note that dilation appears to be more difficult, and if small values of $m$ are used, there may be more distortion in the embeddings than for more simple articulations.

In [ ]:
#### Parameters ####
N = 9       # Number of data points
partial = 1  # Subsampling parameter of data points (if desired)
d = 2        # ambient data dimension
# m = 1000      # Number of samples of reference measure
# n = 2500       # Number of samples of data measures
mlist = np.arange(500,2100,250)
nlist = 2500
numiter = 10  # Number of iterations (each trial of the LOT Wassmap algorithm is run numiter times and error is averaged)

lambd = 1e1    # entropic regularization parameter

#### Generate Reference Measure ####
## Reference measure is an empirical sample of N(0,I)

a, b = np.ones((m,)) / m, np.ones((n,)) / n  # uniform distribution on samples

mu_r = np.array([0, 0])   # mean 0
#cov_r = .1*np.array([[1, -.5], [-.5, 1]])  # reference covariance matrix
cov_r = np.array([[1, 0], [0, 1]])

Y = np.zeros((N//partial, 2)) 

noise_level = 0.5 #.5 # noise level (standard deviation) added to covariance matrix of data measures

mu_t_list = []
cov_t_list = []

scaling = np.linspace(1, 4, num=int(np.sqrt(N)))
scaling = np.flip(scaling)
XX, YY = np.meshgrid(scaling,scaling)
XX = XX.reshape(-1)
YY = YY.reshape(-1)

for i in range(N//partial):
    mu_t = np.array([0, 0])
    #cov_t = np.array([[1, -.5], [-.5, 1]])
    cov_t = np.array([[1, 0], [0, 1]])
    errMat = noise_level * np.random.rand(d,d)
    cov_t = cov_t + errMat @ np.transpose(errMat)
    cov_t = np.diag([XX[i],YY[i]])@cov_t@np.diag([XX[i],YY[i]])
    
    mu_t_list.append(mu_t)
    cov_t_list.append(cov_t)

    Y[i,0] = XX[i] - np.mean(XX)
    Y[i,1] = YY[i] - np.mean(YY)

errStore_dilation = np.zeros((len(mlist), numiter))
errStoreEntropic_dilation = np.zeros((len(mlist), numiter))


for repiter in range(numiter):
    for miter in range(len(mlist)):

        m = mlist[miter]
        n = m

        lot_WassMap = LOT_WassMap_Gauss(m, n, N, mu_r, cov_r, mu_t_list,
                        cov_t_list, Y=Y, partial=partial, lambd = 1e2,figs=False)
        embedding, entropic_embedding, err, entropic_error = lot_WassMap.calc_LOT_WassMap()    

        errStore_dilation[miter,repiter] = err
        errStoreEntropic_dilation[miter,repiter] = entropic_error

errMean_dilation = np.mean(errStore_dilation,axis=1)
errStd_dilation = np.std(errStore_dilation,axis=1)

Entropic_errorMean_dilation = np.mean(errStoreEntropic_dilation,axis=1)
Entropic_errorStd_dilation = np.std(errStoreEntropic_dilation,axis=1)

fig = pl.figure(figsize=(12,5))
ax = fig.add_subplot(121)
ax.plot(mlist,errMean_dilation,'b',marker='o')
ax.fill_between(mlist,errMean_dilation - errStd_dilation,errMean_dilation + errStd_dilation,color = 'gray',alpha=.2)
ax.set_title('Linear Program')
ax.set_xticks(mlist)
ax.set_xlabel('Sample size $m$')
ax.set_ylabel('Relative Error')
# ax.set_aspect('equal')

ax2 = fig.add_subplot(122)
ax2.plot(mlist,Entropic_errorMean_dilation,'b',marker='o')
ax2.fill_between(mlist,Entropic_errorMean_dilation - Entropic_errorStd_dilation,Entropic_errorMean_dilation + Entropic_errorStd_dilation,color = 'gray',alpha=.2)
ax2.set_title('Sinkhorn')
ax2.set_xticks(mlist)
ax2.set_xlabel('Sample size $m$')
ax2.set_ylabel('Relative Error')
#ax2.set_aspect('equal')
fig.tight_layout(pad=5.0)
pl.show()


## Timing Experiment
Experiment 5 and Figures 9 and 10 in the paper. Here we use the grid translation example to time traditional Wassmap against LOT Wassmap using both the linear program and Sinkhorn solvers to compute the optimal tranpsort maps. The first cell times LP, Sinkhorn with $\lambda=1$ and Wassmap. The second cell times LP and Sinkhorn with $\lambda=1$ and $\lambda=10$.

In [ ]:
#### Parameters ####
N = 25       # Number of data points
partial = 1  # Subsampling parameter of data points (if desired)
d = 2        # ambient data dimension
# m = 1000      # Number of samples of reference measure
# n = m        # Number of samples of data measures
mlist = np.arange(100,850,100)
nlist = mlist
numiter =10  # Number of iterations (each trial of the LOT Wassmap algorithm is run numiter times and error is averaged)

lambd = 1e1    # entropic regularization parameter

#### Generate Reference Measure ####
## Reference measure is an empirical sample of N(0,I)

a, b = np.ones((m,)) / m, np.ones((n,)) / n  # uniform distribution on samples

mu_r = np.array([0, 0])   # mean 0
cov_r = np.array([[1, 0], [0, 1]])  # Identity covariance matrix


Y = np.zeros((N//partial, 2)) 

noise_level = 0.5 # noise level (standard deviation) added to covariance matrix of data measures

mu_t_list = []
cov_t_list = []

# form coordinate grid for the 2-d translation lattice
x = np.linspace(-10, 10, int(np.sqrt(N)))
y = np.linspace(-10, 10, int(np.sqrt(N)))
# full coordinate arrays
xx, yy = np.meshgrid(x, y)
xx = xx.reshape(-1)
yy = yy.reshape(-1)

for i in range(N//partial):
    mu_t = np.array([xx[i], yy[i]]) # mean on the sampled 2-D lattice
    cov_t = np.array([[1, -.5], [-.5, 1]]) # correlated covariance matrix
    errMat = noise_level * np.random.rand(d,d) # random Gaussian noise
    cov_t = cov_t + errMat @ np.transpose(errMat) # Add symmetric noise to covariance matrix
    
    mu_t_list.append(mu_t)
    cov_t_list.append(cov_t)
    
    Y[i,:] = mu_t
    
timeStore_2dtrans = np.zeros((len(mlist), numiter))
timeStoreEntropic_2dtrans = np.zeros((len(mlist), numiter))
timeStoreMDS_2dtrans = np.zeros((len(mlist), numiter))

for repiter in range(numiter):
    for miter in range(len(mlist)): 

            m = mlist[miter]
            n = m
            
            lot_WassMap = LOT_WassMap_Gauss(m, n, N, mu_r, cov_r, mu_t_list,
                cov_t_list, Y=Y, partial=partial,figs=False,lambd=lambd)
            embedding, LP_time, entropic_embedding, entropic_time, MDS_embedding, MDS_time = lot_WassMap.LOT_timing()
            
            timeStore_2dtrans[miter,repiter] = LP_time
            timeStoreEntropic_2dtrans[miter,repiter] = entropic_time
            timeStoreMDS_2dtrans[miter,repiter] = MDS_time

timeMean_2dtrans = np.mean(timeStore_2dtrans, axis = 1)
timeStd_2dtrans = np.std(timeStore_2dtrans, axis = 1)

Entropic_timeMean_2dtrans = np.mean(timeStoreEntropic_2dtrans, axis = 1)
Entropic_timeStd_2dtrans = np.std(timeStoreEntropic_2dtrans, axis = 1)

MDS_timeMean_2dtrans = np.mean(timeStoreMDS_2dtrans, axis = 1)
MDS_timeStd_2dtrans = np.std(timeStoreMDS_2dtrans, axis = 1)


fig = pl.figure(figsize=(12,5))
ax = fig.add_subplot(111)
line1, = ax.plot(mlist,timeMean_2dtrans,color = 'cyan',marker='o')
ax.fill_between(mlist,timeMean_2dtrans - timeStd_2dtrans,timeMean_2dtrans + timeStd_2dtrans,color = 'cyan',alpha=.1)
line2, = ax.plot(mlist,Entropic_timeMean_2dtrans,color = 'magenta',marker='o')
ax.fill_between(mlist,Entropic_timeMean_2dtrans - Entropic_timeStd_2dtrans,Entropic_timeMean_2dtrans + Entropic_timeStd_2dtrans,color = 'magenta',alpha=.1)
line3, = ax.plot(mlist,MDS_timeMean_2dtrans,color='red',marker='o')
ax.fill_between(mlist,MDS_timeMean_2dtrans - MDS_timeStd_2dtrans,MDS_timeMean_2dtrans + MDS_timeStd_2dtrans,color = 'red',alpha=.1)

ax.set_title('Timing')
ax.set_xticks(mlist)
ax.set_xlabel('Sample size $m$')
ax.set_ylabel('(Time) (s)')
ax.legend([line1,line2,line3],["LP","Sinkhorn ($\lambda=1$)","Wassmap"])
pl.show()

Repeat of previous experiment for timing only LOT Wassmap with LP and Sinkhorn for various $\lambda$.

In [ ]:
#### Parameters ####
N = 25       # Number of data points
partial = 1  # Subsampling parameter of data points (if desired)
d = 2        # ambient data dimension
# m = 1000      # Number of samples of reference measure
# n = m        # Number of samples of data measures
mlist = np.arange(1000,2000,100)
nlist = mlist
numiter =10  # Number of iterations (each trial of the LOT Wassmap algorithm is run numiter times and error is averaged)

#lambd = 1e1    # entropic regularization parameter

#### Generate Reference Measure ####
## Reference measure is an empirical sample of N(0,I)

a, b = np.ones((m,)) / m, np.ones((n,)) / n  # uniform distribution on samples

mu_r = np.array([0, 0])   # mean 0
cov_r = np.array([[1, 0], [0, 1]])  # Identity covariance matrix


Y = np.zeros((N//partial, 2)) 

noise_level = 0.5 # noise level (standard deviation) added to covariance matrix of data measures

mu_t_list = []
cov_t_list = []

# form coordinate grid for the 2-d translation lattice
x = np.linspace(-10, 10, int(np.sqrt(N)))
y = np.linspace(-10, 10, int(np.sqrt(N)))
# full coordinate arrays
xx, yy = np.meshgrid(x, y)
xx = xx.reshape(-1)
yy = yy.reshape(-1)

for i in range(N//partial):
    mu_t = np.array([xx[i], yy[i]]) # mean on the sampled 2-D lattice
    cov_t = np.array([[1, -.5], [-.5, 1]]) # correlated covariance matrix
    errMat = noise_level * np.random.rand(d,d) # random Gaussian noise
    cov_t = cov_t + errMat @ np.transpose(errMat) # Add symmetric noise to covariance matrix
    
    mu_t_list.append(mu_t)
    cov_t_list.append(cov_t)
    
    Y[i,:] = mu_t
    

timeStore_rotation_LP = np.zeros((len(mlist), numiter))
# timeStoreEntropic_rotation_lambda01 = np.zeros((len(mlist), numiter))
timeStoreEntropic_rotation_lambda1 = np.zeros((len(mlist), numiter))
timeStoreEntropic_rotation_lambda10 = np.zeros((len(mlist), numiter))
# timeStoreEntropic_rotation_lambda100 = np.zeros((len(mlist), numiter))
# timeStoreEntropic_rotation_lambda1000 = np.zeros((len(mlist), numiter))

for repiter in range(numiter):
    for miter in range(len(mlist)): 

            m = mlist[miter]
            n = m
            
            lot_WassMap = LOT_WassMap_Gauss(m, n, N, mu_r, cov_r, mu_t_list,
                cov_t_list, Y=Y, partial=partial,figs=False)
            embedding, LP_time = lot_WassMap.LOT_timing_LP()
            timeStore_rotation_LP[miter,repiter] = LP_time
            
#             entropicembedding, Sinkhorn_time_lambda01 = lot_WassMap.LOT_timing_Sinkhorn(lambd = .1)
#             timeStoreEntropic_rotation_lambda01[miter, repiter] = Sinkhorn_time_lambda01
            
            entropicembedding, Sinkhorn_time_lambda1 = lot_WassMap.LOT_timing_Sinkhorn(lambd = 1)
            timeStoreEntropic_rotation_lambda1[miter, repiter] = Sinkhorn_time_lambda1
            
            entropicembedding, Sinkhorn_time_lambda10 = lot_WassMap.LOT_timing_Sinkhorn(lambd = 10)
            timeStoreEntropic_rotation_lambda10[miter, repiter] = Sinkhorn_time_lambda10
            
#             entropicembedding, Sinkhorn_time_lambda100 = lot_WassMap.LOT_timing_Sinkhorn(lambd = 100)
#             timeStoreEntropic_rotation_lambda100[miter, repiter] = Sinkhorn_time_lambda100
            
#             entropicembedding, Sinkhorn_time_lambda1000 = lot_WassMap.LOT_timing_Sinkhorn(lambd = 1000)
#             timeStoreEntropic_rotation_lambda1000[miter, repiter] = Sinkhorn_time_lambda1000
            

timeMean_rotation_LP = np.mean(timeStore_rotation_LP, axis = 1)
timeStd_rotation_LP = np.std(timeStore_rotation_LP, axis = 1)

# Entropic_timeMean_rotation_lambda01 = np.mean(timeStoreEntropic_rotation_lambda01, axis = 1)
# Entropic_timeStd_rotation_lambda01 = np.std(timeStoreEntropic_rotation_lambda01, axis = 1)

Entropic_timeMean_rotation_lambda1 = np.mean(timeStoreEntropic_rotation_lambda1, axis = 1)
Entropic_timeStd_rotation_lambda1 = np.std(timeStoreEntropic_rotation_lambda1, axis = 1)

Entropic_timeMean_rotation_lambda10 = np.mean(timeStoreEntropic_rotation_lambda10, axis = 1)
Entropic_timeStd_rotation_lambda10 = np.std(timeStoreEntropic_rotation_lambda10, axis = 1)

# Entropic_timeMean_rotation_lambda100 = np.mean(timeStoreEntropic_rotation_lambda100, axis = 1)
# Entropic_timeStd_rotation_lambda100 = np.std(timeStoreEntropic_rotation_lambda100, axis = 1)

# Entropic_timeMean_rotation_lambda1000 = np.mean(timeStoreEntropic_rotation_lambda1000, axis = 1)
# Entropic_timeStd_rotation_lambda1000 = np.std(timeStoreEntropic_rotation_lambda1000, axis = 1)


c = pl.cm.tab10(np.linspace(0,1,65))
fig = pl.figure(figsize=(12,5))
ax = fig.add_subplot(111)
line11, = ax.plot(mlist,timeMean_rotation_LP,color = 'cyan', marker='o')
ax.fill_between(mlist,timeMean_rotation_LP - timeStd_rotation_LP,timeMean_rotation_LP + timeStd_rotation_LP,color = 'cyan',alpha=.1)

# line12, = ax.plot(mlist,Entropic_timeMean_rotation_lambda01,color='red',marker='o')
# ax.fill_between(mlist,Entropic_timeMean_rotation_lambda01 - Entropic_timeStd_rotation_lambda01,Entropic_timeMean_rotation_lambda01 + Entropic_timeStd_rotation_lambda01,color = 'red',alpha=.1)

line13, = ax.plot(mlist,Entropic_timeMean_rotation_lambda1,color='red',marker='o')
ax.fill_between(mlist,Entropic_timeMean_rotation_lambda1 - Entropic_timeStd_rotation_lambda1,Entropic_timeMean_rotation_lambda1 + Entropic_timeStd_rotation_lambda1,color = 'red',alpha=.1)

line14, = ax.plot(mlist,Entropic_timeMean_rotation_lambda10,color = 'magenta',marker='o')
ax.fill_between(mlist,Entropic_timeMean_rotation_lambda10 - Entropic_timeStd_rotation_lambda10,Entropic_timeMean_rotation_lambda10 + Entropic_timeStd_rotation_lambda10,color = 'magenta',alpha=.1)

# line15, = ax.plot(mlist,Entropic_timeMean_rotation_lambda100,color='green',marker='o')
# ax.fill_between(mlist,Entropic_timeMean_rotation_lambda100 - Entropic_timeStd_rotation_lambda100,Entropic_timeMean_rotation_lambda100 + Entropic_timeStd_rotation_lambda100,color = 'green',alpha=.1)

# line16, = ax.plot(mlist,Entropic_timeMean_rotation_lambda1000,color = 'magenta',marker='o')
# ax.fill_between(mlist,Entropic_timeMean_rotation_lambda1000 - Entropic_timeStd_rotation_lambda1000,Entropic_timeMean_rotation_lambda1000 + Entropic_timeStd_rotation_lambda1000,color = 'magenta',alpha=.1)

ax.set_title('Timing')
ax.set_xticks(mlist)
ax.set_xlabel('Sample size $m$')
ax.set_ylabel('Time (s)')
ax.legend([line11,line13,line14],["LP", "$\lambda = 1$", "$\lambda = 10$"])

pl.show()